In [ ]:
!pip install scipy==1.1.0

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('caption.py', 'wb').write(src)

In [ ]:
src = list(files.upload().values())[0]
open('models.py', 'wb').write(src)

In [ ]:
import caption
import torch
import json
import os
from collections import OrderedDict
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load('/content/drive/MyDrive/data/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar', map_location=str(device))
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

# Load word map (word2ix)
with open('/content/drive/MyDrive/data/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json', 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

In [ ]:
path_dir = '/content/drive/MyDrive/data/aliceintotherabbithole'
file_list = os.listdir(path_dir)

alice_data = []
for img_name in file_list:
  try:
    Image.open(path_dir+'/'+img_name) 
  except:
    continue

  # Encode, decode with attention and beam search
  seq, alphas = caption.caption_image_beam_search(encoder, decoder, path_dir+'/'+img_name, word_map, 3)
  alphas = torch.FloatTensor(alphas)

  # Visualize caption and attention of best sequence
  words = caption.visualize_att(path_dir+'/'+img_name, seq, alphas, rev_word_map, smooth=True)
  data = {'file_name': img_name, 'caption': words}
  alice_data.append(data)
  print(len(alice_data))

In [ ]:
json_data = OrderedDict()
json_data['alice'] = alice_data
with open('alice.json', 'w', encoding="utf-8") as make_file:
    json.dump(json_data, make_file, ensure_ascii=False)